In [1]:
import os
import json
import re
import pandas as pd

company_info = pd.read_csv('/home/ljh/RGEN_A/original_data/company_info_4112.csv')

full_name = list(company_info['fullname'])
short_name = list(company_info['name'])
# 去除后缀的代码
# codes = list(company_info['ts_code'].apply(lambda x: x.split('.')[0]))
codes = list(company_info['ts_code'])
# 关系列表
relations = ['合作','投资','收购','供应商','技术转让']

# Function to extract triples from content
def extract_triples(content):
    if "无关联关系" not in content:
        # 提取其中所有三元组转化为元组变量
        new_triples = []
        triples = re.findall(r'\(.*?\)', content)
        for triple in triples:
            triple = triple.replace(' ', '')
            triple = triple[1:-1]
            triple_elements = triple.split(',')
            if len(triple_elements) == 3:
                new_triples.append([triple_elements[0], triple_elements[1], triple_elements[2]])
            triple_elements = triple.split('，')
            if len(triple_elements) == 3:
                new_triples.append([triple_elements[0], triple_elements[1], triple_elements[2]])
        return new_triples
    return []

# Directory containing the jsonl files
directory = '/home/ljh/RGEN_B/construction/CRG/news/response'
# Read all jsonl files and extract triples
for filename in os.listdir(directory):
    # Extract year from filename
    year = filename.split('_')[0]
    if filename.endswith('.jsonl'):
        filepath = os.path.join(directory, filename)
        with open(filepath, 'r') as infile:
            for line in infile:
                data = json.loads(line)
                content = data['response']['body']['choices'][0]['message']['content']
                triples = extract_triples(content)
                for triple in triples:
                    sub_triple = []
                    if triple[0] in full_name or triple[0] in short_name or triple[0] in codes:
                        if triple[2] in full_name or triple[2] in short_name or triple[2] in codes:
                            # 将三元组中的实体triple[0]和triple[2]替换为公司代码
                            if triple[0] in codes:
                                pass
                            elif triple[0] in full_name:
                                triple[0] = codes[full_name.index(triple[0])]
                            elif triple[0] in short_name:
                                triple[0] = codes[short_name.index(triple[0])]
                            
                            if triple[2] in codes:
                                pass
                            elif triple[2] in full_name:
                                triple[2] = codes[full_name.index(triple[2])]
                            elif triple[2] in short_name:
                                triple[2] = codes[short_name.index(triple[2])]
                            # 检索relations中哪些关系存在triple[1]中，所有存在的关系生成新的三元组
                            for relation in relations:
                                if relation in triple[1]:    
                                    sub_triple.append(triple[0] + '\t' + relation + '\t' + triple[2])
                    if year == '2023':
                        with open('extracted_triples_codes_2023.txt', 'a') as year_file:
                            for triple in sub_triple:
                                year_file.write(triple + '\n')
                    elif year == '2024':
                        with open('extracted_triples_codes_2024.txt', 'a') as year_file:
                            for triple in sub_triple:
                                year_file.write(triple + '\n')

In [2]:
# 去除txt中重复的三元组
with open('extracted_triples_codes_2023.txt', 'r') as infile:
    lines = infile.readlines()
    lines = list(set(lines))
    with open('extracted_triples_codes_2023_dup.txt', 'w') as outfile:
        for line in lines:
            outfile.write(line)
with open('extracted_triples_codes_2024.txt', 'r') as infile:
    lines = infile.readlines()
    lines = list(set(lines))
    with open('extracted_triples_codes_2024_dup.txt', 'w') as outfile:
        for line in lines:
            outfile.write(line)

In [10]:
import re
# Function to extract triples from content
def extract_triples(content):
    if "无关联关系" not in content:
        # 提取其中所有三元组转化为元组变量
        new_triples = []
        triples = re.findall(r'\(.*?\)', content)
        for triple in triples:
            triple = triple.replace(' ', '')
            triple = triple[1:-1]
            triple_elements = triple.split(',')
            if len(triple_elements) == 3:
                new_triples.append((triple_elements[0], triple_elements[1], triple_elements[2]))
            triple_elements = triple.split('，')
            if len(triple_elements) == 3:
                new_triples.append((triple_elements[0], triple_elements[1], triple_elements[2]))
        return new_triples
    return []

extract_triples('(渤海租赁, 投资收购, Avolon)\n(Avolon, 供应商, 波音)\n(Avolon, 供应商, 空客)\n(渤海租赁, 合作, Avolon)')
# l = extract_triples('(渤海租赁, 投资收购, Avolon)\n(Avolon, 供应商, 波音)\n(Avolon, 供应商, 空客)\n(渤海租赁, 合作, Avolon)')
# print(type(l[0]))

[('渤海租赁', '投资收购', 'Avolon'),
 ('Avolon', '供应商', '波音'),
 ('Avolon', '供应商', '空客'),
 ('渤海租赁', '合作', 'Avolon')]